# Creo un nuovo dataset 


---
## Analisi dei commenti di un ristorante

### Luini Panzerotti - Milano

URL di riferimento: https://www.tripadvisor.it/Restaurant_Review-g187849-d1097473-Reviews-Luini-Milan_Lombardy.html
Home --> Ristoranti --> Milano

In [1]:
# IMPORT NECESSARI
import requests
from bs4 import BeautifulSoup

print("Imported")

Imported


In [2]:
page_URL = "https://www.tripadvisor.it/Restaurant_Review-g187849-d1097473-Reviews-Luini-Milan_Lombardy.html"

data = requests.get(page_URL)
clear_data = BeautifulSoup(data.content, "lxml")

print("Done")

Done


---
### STRUTTURA HTML DEL SITO 

#### REVIEW
`<p class="partial_entry"> qui c'è la review </p>`

#### N° TOTALE DI REVIEWS
`<a data-page-number="25" ...>
    25
 </a>`
 
#### NAMES
`<span class="expand_inline scrname" …>AlexandraCapeTown</span>`

#### RATINGS
`<span class="ui_bubble_rating bubble_30"></span>`

---
### PRIMO SCRAPING (sulla prima pagina)

Ottengo i nomi, i commenti, le review e da dove è stato scritto il commento per quanto rigurda la prima pagina. Ciò è necessario, in quanto, dalla seconda in poi l'URL cambia, permettendoci di iterare.

In [3]:
# OTTENGO UNA LISTA DI DIV CHE CONTENGONO NOME, RECENSIONE E VOTO

all_reviews = []
all_names = []
all_ratings = []
all_mobile = []

review_container = clear_data.findAll("div", attrs={"class":"review-container"})

for container in review_container:
  
    ### REVIEW ###
    review = container.find('p', attrs={'class': 'partial_entry'})
    if review:
        all_reviews.append(review.text)
        
    ### NAME ###
    name = container.find('span', attrs={'class': ('expand_inline', 'scrname')})
    if name:
      all_names.append(name.text)
    else:
      all_names.append('NO_NAME')
    
    ### RATING ###
    rating = container.find('span', attrs={'class': 'ui_bubble_rating'})
    all_ratings.append(rating['class'][1][7:8])
    
    ### MOBILE ###
    mobile = container.find('span', attrs={'class': 'viaMobile'})
    if mobile:
      all_mobile.append('Mobile')
    else:
      all_mobile.append('PC')
    
    
################################################################################################################################
        
print("Executed")
print('Nomi: ', len(all_names))
print('Commenti: ', len(all_reviews))
print('Voti: ', len(all_ratings))
print('Scritto da: ', len(all_ratings))

Executed
Nomi:  10
Commenti:  10
Voti:  10
Scritto da:  10


### SECONDO SCRAPING (dalla seconda pagina in poi)

Notiamo dall'URL che è possibile iterare sulle pagine, ottenendo i dati!

In [4]:
from tqdm import tqdm

URL_COMPLETO = 'https://www.tripadvisor.it/Restaurant_Review-g187849-d1097473-Reviews-or20-Luini-Milan_Lombardy.html'

URL_p1 = 'https://www.tripadvisor.it/Restaurant_Review-g187849-d1097473-Reviews-or'
URL_p2 = '0-Luini-Milan_Lombardy.html'
    
# OTTENGO I COMMENTI, NOMI E VOTI DALLA SECONDA PAGINA IN POI

total_pages = int(clear_data.select("a.pageNum.last.taLnk")[0].text) # 25 come intero

# per iterare dobbiamo partire dalla seconda pagina dei commenti, così nell'URL compaiono le cifre che ci servono
for r in tqdm(range(1,total_pages)):

    # OTTENGO L'URL DELLA PAGINA
    URL_temp = URL_p1 + str(r) + URL_p2

    # OTTENGO I DATI E CI APPLICO BEATIFULSOUP
    data = requests.get(URL_temp)
    clear_data = BeautifulSoup(data.content, "lxml")
    

    review_container = clear_data.findAll("div", attrs={"class":"review-container"})

    for container in review_container:
  
      ### REVIEW ###
      review = container.find('p', attrs={'class': 'partial_entry'})
      if review:
          all_reviews.append(review.text)

      ### NAME ###
      name = container.find('span', attrs={'class': ('expand_inline', 'scrname')})
      if name:
        all_names.append(name.text)
      else:
        all_names.append('NO_NAME')

      ### RATING ###
      rating = container.find('span', attrs={'class': 'ui_bubble_rating'})
      all_ratings.append(rating['class'][1][7:8])

      ### MOBILE ###
      mobile = container.find('span', attrs={'class': 'viaMobile'})
      if mobile:
        all_mobile.append('Mobile')
      else:
        all_mobile.append('PC')
    
        
################################################################################################################################
        
print("Executed")
print('total_pages: ', total_pages)
print('Nomi: ', len(all_names))
print('Commenti: ', len(all_reviews))
print('Voti: ', len(all_ratings))
print('Scritto da: ', len(all_mobile))

100%|██████████| 1318/1318 [20:09<00:00,  1.24it/s]

Executed
total_pages:  1319
Nomi:  13185
Commenti:  13185
Voti:  13185
Scritto da:  13185


In [0]:
# display(all_reviews)

In [0]:
# print(all_names)

In [0]:
#print(all_ratings, '\n')
#print(len(all_ratings))

In [0]:
#display(all_mobile)

#### Problema recensioni troppo lunghe
Alla fine di alcune recensioni troviamo un '...Più', che non possiamo espandere. Data la mole di dati, decidiamo dunque di non considerarle.

In [9]:
all_reviews_filtered = []
counter_reviews_scartate = 0;
counter_reviews_da_usare = 0;

for review in all_reviews:
    if '...Più' in review:
        all_reviews_filtered.append('RECENSIONE_TROPPO_LUNGA')
        counter_reviews_scartate+=1;
    else:
        all_reviews_filtered.append(review)
        counter_reviews_da_usare+=1;
        
        
print(
    'Totale recensioni:', counter_reviews_scartate + counter_reviews_da_usare, '\n',
    'Recensioni analizzabili:', counter_reviews_da_usare, '\n',
    'Recensioni scartate:', counter_reviews_scartate, '\n')

display(all_reviews_filtered[:10])

Totale recensioni: 13185 
 Recensioni analizzabili: 8583 
 Recensioni scartate: 4602 



["Per chi va in centro a Milano non puo' non fermarsi ad assaggiare un panzerotto da Luini, a Milano diventato oramai un' Istituzione. I' ho preso con prosciutto e mozzarella. Davvero molto buono.",
 "L'ho conosciuto solo pochi giorni fa... e ci tornerò sicuramente. Ci sono panzerotti per tutti i gusti, sia al forno che fritti, e il bello è mangiarli nel sacchettino di carta fra i vicoli circostanti, in mezzo a tutti gli altri clienti...",
 'A parte la coda,Sempre lunghissima!Ma ne vale la pena!\nÈ sempre un piacere per le papille gustative!!\nOgni volta è tappa fissa!!!\nDa provare',
 'RECENSIONE_TROPPO_LUNGA',
 "Sempre il miglio panzerotto, ma non mi sembra più così buono. Forse perchè sono passati 40 anni da quando bigiavo al liceo, Difficile comunque trovare difetti. E' buonissimo, magari frli un pò più grandi...",
 'Panzerotto molto buono con ingredienti di qualità nonostante la grande quantità di clienti.\nFila scorrevole e poca attesa in proporzione alla gente',
 'RECENSIONE_TRO

---
### Analisi dei nomi
Cerchiamo di capire se chi ha commentato è un uomo (M) o una donna (F).

#### Elimino spazi e numeri
Ho bisogno di tenere solo le informazioni riguardanti il nome, elimino il resto.

In [10]:
import re

all_names_filtered = []

regex = re.compile('[^a-zA-Z]')

for name in all_names:
    
    if ' ' in name:                           # se c'è uno spazio nel nome
        space = name.index(' ')               # ne trovo l'indice
        name = regex.sub('', name[:space])    # effettuo un controllo su tutto ciò che non è alfabeto fino a quell'indice
        all_names_filtered.append(name)       # e aggiungo la stringa
        
    else:
        name = regex.sub('', name)            # in caso contrario effettuo un controllo su tutto ciò che non è alfabeto
        all_names_filtered.append(name)       # e aggiungo la stringa
        
print(all_names, '\n')
print(all_names_filtered)

['andrea g', 'AnnyS347', 'mgeg10', 'naturelle09', 'Davide F', '326danielee', 'joe_in_the_city', 'Stefania C', 'alessio a', 'ReviewsItaly', 'Eugenix2', 'allovertheworldd', 'Alberto A', 'feddy391', 'Silvana S', 'ArmandoAnna', 'Robi R', 'Tamara M', 'clou1104', '624mariapiaa', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'giorgio c', 'link7910', 'giuseppelorenzo', 'CateCate969', 'Emanuele_Q', '301DavidG', 'marinabb2016', 'Giorgio T', 'Cinzia P', 'lucalisci', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'Y8518INangelop', 'Marghille', 'Diletta A', 'Checchina85', 'Thaumasios', 'Viatrix81', '583evitat', 'alessiabA9920MK', 'Francessscaa', 'spike70', '

#### Utilizzo gender_guesser
Infine, posso utilizzare la libreria gender_guesser.

NB: Genderize ti permette di analizzare 1000 nomi al giorno.

In [11]:
!pip install gender_guesser

    100% |████████████████████████████████| 389kB 5.3MB/s 


In [12]:
import gender_guesser.detector as gender
d = gender.Detector()

names_with_gender = []

for name in all_names_filtered:
    temp_gender = d.get_gender(name)
    names_with_gender.append(temp_gender)
  
print(names_with_gender)

['unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'female', 'unknown', 'male', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 

In [13]:
all_genders = []
i = 0

for guess in names_with_gender:
    if guess == 'male' or guess == 'mostly_male':
        all_genders.append('M')
    elif guess == 'female' or guess == 'mostly_female':
        all_genders.append('F')
    else:
        all_genders.append('UNKNOWN')

    
print(all_genders)

['UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'F', 'UNKNOWN', 'M', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M',

---
## Salviamo ora i dati ottenuti in una tabella Excel
Così non dovremo ogni volta rieseguire tutto il codice



### Controlliamo che ci siano lo stesso numero di dati
Potremo così creare una tabella sfruttando Pandas

In [14]:
print('Nomi: ', len(all_names_filtered))
print('Commenti: ', len(all_reviews_filtered))
print('Voti: ', len(all_ratings))
print('Mobile: ', len(all_mobile))

Nomi:  13185
Commenti:  13185
Voti:  13185
Mobile:  13185


### Eliminiamo le recensioni troppo lunghe e quelle di cui non sappiamo il genere del commentatore
Creo una lista di liste [ ['NOME', 'GENERE', 'RECENSIONE', 'VOTO'], ['NOME', 'GENERE', 'RECENSIONE', 'VOTO'], ... ]

In [15]:
complete_list = []

i = len(all_names_filtered)

for indice in range(i):
  if all_genders[indice] != 'UNKNOWN' and all_reviews_filtered[indice] != 'RECENSIONE_TROPPO_LUNGA':
    temp = [all_names_filtered[indice], all_genders[indice], all_reviews_filtered[indice], all_ratings[indice], all_mobile[indice]]
    complete_list.append(temp)
    
print('DATI ANALIZZABILI: ', len(complete_list))
display(complete_list[:5])


DATI ANALIZZABILI:  1781


[['Davide',
  'M',
  "Sempre il miglio panzerotto, ma non mi sembra più così buono. Forse perchè sono passati 40 anni da quando bigiavo al liceo, Difficile comunque trovare difetti. E' buonissimo, magari frli un pò più grandi...",
  '3',
  'PC'],
 ['Stefania',
  'F',
  'Ogni volta che passo dal centro..mangio sempre un paio di panzerotti....super buoni. Io prendo quasi sempre quelli mozzarella e pomodoro',
  '4',
  'Mobile'],
 ['Alberto',
  'M',
  'Fila chilometrica ma ne è valsa la pena Come sempre una tradizione da non dimenticare ottimo il prodotto',
  '5',
  'Mobile'],
 ['Giorgio',
  'M',
  'Sempre un piacere gustare un paio di buoni panzerotti ( mozzarella/ pomodoro e mozzarella cipolle olive ) da Luini. Coda veloce, personale gentile, purtroppo bisogna mangiare in piedi...',
  '4',
  'PC'],
 ['Cinzia',
  'F',
  'Per chi si trova a dover pranzare e vuole gustarsi un ottimo panzerotto, non deve dimenticarsi di fare tappa da Luini.\nUn pò di coda... ma ne vale la pena.',
  '4',
  'P

### Creiamo la tabella

In [18]:
 !pip install pandas
 !pip install xlsxwriter
import pandas as pd

    100% |████████████████████████████████| 143kB 4.3MB/s 


In [0]:
# Create a Pandas dataframe from the data.
df = pd.DataFrame({'Nomi': [elemento[0] for elemento in complete_list], 
                   'Genere': [elemento[1] for elemento in complete_list], 
                   'Recensioni': [elemento[2] for elemento in complete_list],
                   'Voti': [elemento[3] for elemento in complete_list],
                   'Scritta da': [elemento[4] for elemento in complete_list]})

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('RISTORANTE_21.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [20]:
partial_dataframe = pd.read_excel("RISTORANTE_21.xlsx")

display(partial_dataframe)

,Genere,Nomi,Recensioni,Scritta da,Voti
0,M,Davide,"Sempre il miglio panzerotto, ma non mi sembra ...",PC,3
1,F,Stefania,Ogni volta che passo dal centro..mangio sempre...,Mobile,4
2,M,Alberto,Fila chilometrica ma ne è valsa la pena Come s...,Mobile,5
3,M,Giorgio,Sempre un piacere gustare un paio di buoni pan...,PC,4
4,F,Cinzia,Per chi si trova a dover pranzare e vuole gust...,PC,4
5,F,Diletta,Mangerete i migliori panzerotti in assoluto. I...,Mobile,4
6,F,Elisa,Non ci sono parole per descrivere la bontà dei...,Mobile,5
7,M,Antonio,Ci siamo stati il 7/12 di passaggio dopo una s...,Mobile,5
8,M,Giancarlo,"qualità ottima, un pò di fila, ma ne vale la p...",PC,5
9,M,Roberto,che dire di luini.....\nsono più di 30 anni ch...,PC,5
